<a href="https://colab.research.google.com/github/ellreece/cs065/blob/main/ACTS%20190.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import pandas as pd
from datetime import datetime
import numpy as np

In [3]:
# all dates directories
fol_names = os.listdir('/content/drive/MyDrive/Senior Year/ACTS 190/Sample Data')
fol_names.remove('summary_data.csv')
fol_names.remove('vehicle_data.csv')

In [4]:
# initial dataset
agg_data = pd.read_csv('/content/drive/MyDrive/Senior Year/ACTS 190/Sample Data/'+ fol_names[0] + '/claim.csv' )
agg_data['date'] = fol_names[0]


In [5]:
# appending datasets 
for dir in fol_names[1:]:
    dat_path = '/content/drive/MyDrive/Senior Year/ACTS 190/Sample Data/' + dir + '/claim.csv' 
    fol_data = pd.read_csv(dat_path)
    fol_data['date'] = dir
    agg_data = pd.concat([agg_data, fol_data])

In [6]:
agg_data.head()

,claim_id,bodily_injury_count,claim_status,line_of_business,loss_cause,detailed_loss_cause,loss_datetime,reported_datetime,close_datetime,reopen_datetime,loss_payment_amount,number_of_vehicles_on_claim,reported_by,loss_county,loss_state,date,insured_fault_pct
0,14266403,2,Open,Personal Auto Line,Collision with Motor Vehicle/Watercraft,Sideswipe,2021-10-17 18:55:00,2021-10-18 19:21:30.762,NaN,NaN,9169.56,2,Claimant,Maricopa,AZ,20220222,NaN
1,14500001,1,Closed,Personal Auto Line,Collision with Motor Vehicle/Watercraft,Insured Rear-ended Claimant,2021-11-12 13:53:00,2021-11-13 13:54:12.339,2022-02-21 08:33:46.135,NaN,17036.37,2,Claimant,Maricopa,AZ,20220222,NaN
2,14918321,1,Open,Personal Auto Line,Collision with Motor Vehicle/Watercraft,Front End Collision,2022-01-01 22:32:00,2022-01-01 22:32:54.562,NaN,NaN,14783.39,2,Claimant,Maricopa,AZ,20220222,NaN
3,15520705,2,Open,Personal Auto Line,Collision with Motor Vehicle/Watercraft,Insured Rear-ended Claimant,2022-02-17 18:00:00,2022-02-18 10:55:16.499,NaN,NaN,0.00,2,Spouse,Maricopa,AZ,20220222,NaN
4,8892925,1,Closed,Personal Auto Line,Collision with Motor Vehicle/Watercraft,Collision While Turning,2020-12-02 18:30:00,2020-12-02 22:11:23.580,2022-02-21 12:25:22.940,NaN,22100.00,2,Self,Maricopa,AZ,20220222,NaN


In [7]:
# find out the claims that were reported after the beginning of the observation period (feb 2)
reported_after_feb2 = agg_data.groupby('claim_id', as_index=False).apply(
    lambda x: pd.Series({'reported_after_feb2': pd.to_datetime(x['reported_datetime']).min()>= datetime.strptime('20220202', '%Y%m%d')}))

In [8]:
#Checking Accuracy
reported_after_feb2

,claim_id,reported_after_feb2
0,9037,False
1,12173,False
2,27269,False
3,32589,False
4,34521,False
...,...,...
16347,17469513,True
16348,17509013,True
16349,17513503,True
16350,17513691,True


In [9]:
feasible_claims = reported_after_feb2['claim_id'].loc[reported_after_feb2['reported_after_feb2']]

In [11]:
#Checking Results
feasible_claims

12676    15166491
12686    15169627
12688    15171057
12689    15171295
12690    15171453
           ...   
16346    17457825
16347    17469513
16348    17509013
16349    17513503
16350    17513691
Name: claim_id, Length: 3228, dtype: int64

In [12]:
claim_data_within_window = agg_data.loc[np.isin(agg_data['claim_id'], list(feasible_claims))]

In [13]:
claim_data_within_window

,claim_id,bodily_injury_count,claim_status,line_of_business,loss_cause,detailed_loss_cause,loss_datetime,reported_datetime,close_datetime,reopen_datetime,loss_payment_amount,number_of_vehicles_on_claim,reported_by,loss_county,loss_state,date,insured_fault_pct
3,15520705,2,Open,Personal Auto Line,Collision with Motor Vehicle/Watercraft,Insured Rear-ended Claimant,2022-02-17 18:00:00,2022-02-18 10:55:16.499,NaN,NaN,0.00,2,Spouse,Maricopa,AZ,20220222,NaN
6,15521215,1,Open,Personal Auto Line,Collision with Motor Vehicle/Watercraft,Insured Rear-ended Claimant,2022-02-11 13:58:00,2022-02-18 13:58:45.811,NaN,NaN,0.00,2,Attorney,Maricopa,AZ,20220222,NaN
16,15193065,1,Open,Personal Auto Line,Collision with Motor Vehicle/Watercraft,Sideswipe,2022-02-04 17:50:00,2022-02-04 19:05:18.512,NaN,NaN,179.52,2,Self,NaN,AZ,20220222,NaN
20,15526583,1,Open,Personal Auto Line,Collision with Pedestrian/Bicyclist,Front End Collision,2022-02-15 12:52:00,2022-02-19 12:52:53.710,NaN,NaN,0.00,1,Child,Maricopa,AZ,20220222,NaN
28,15220791,1,Open,Personal Auto Line,Collision with Motor Vehicle/Watercraft,Insured Rear-ended Claimant,2022-02-08 10:00:00,2022-02-08 12:18:34.827,NaN,NaN,10065.86,2,Self,Santa Cruz,CA,20220222,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15,15564843,1,Closed,Personal Auto Line,Collision with Motor Vehicle/Watercraft,Insured Rear-ended Claimant,2022-02-23 07:30:00,2022-02-23 09:34:17.804,2022-08-23 07:32:10.404,NaN,8689.46,2,Claimant,Jackson,MO,20220907,100.0
19,16932867,2,Closed,Personal Auto Line,Collision with Motor Vehicle/Watercraft,Sideswipe,2022-07-05 15:30:00,2022-07-06 09:10:01.793,2022-08-29 08:21:14.960,NaN,12393.67,2,Claimant,NaN,SC,20220907,100.0
21,17208075,1,Closed,Personal Auto Line,Collision with Motor Vehicle/Watercraft,Insured Rear-ended Claimant,2022-07-29 16:15:00,2022-07-29 17:59:38.648,2022-08-30 15:16:45.505,NaN,13865.88,3,Self,NaN,UT,20220907,100.0
22,15246123,1,Open,Personal Auto Line,Collision with Motor Vehicle/Watercraft,Sideswipe,2022-02-10 11:30:00,2022-02-11 07:21:33.203,NaN,2022-09-06 14:16:18.678,1141.88,2,Self,Richmond City,VA,20220907,0.0


In [14]:
claim_data_within_window.groupby
    # sort the records in this group by date (may need to be converted to datetime) in ascending order
    # go from the top of the list to the bottom to look for the first non-zero payment (loss_payment_amount) -> initial payment
    # then look for the last total payment amount


<bound method DataFrame.groupby of     claim_id bodily_injury_count claim_status    line_of_business  \
3   15520705                   2         Open  Personal Auto Line   
6   15521215                   1         Open  Personal Auto Line   
16  15193065                   1         Open  Personal Auto Line   
20  15526583                   1         Open  Personal Auto Line   
28  15220791                   1         Open  Personal Auto Line   
..       ...                 ...          ...                 ...   
15  15564843                   1       Closed  Personal Auto Line   
19  16932867                   2       Closed  Personal Auto Line   
21  17208075                   1       Closed  Personal Auto Line   
22  15246123                   1         Open  Personal Auto Line   
27  15928471                   1       Closed  Personal Auto Line   

                                 loss_cause           detailed_loss_cause  \
3   Collision with Motor Vehicle/Watercraft  Insured Rear-e